In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
#Essentials
import numpy as np
import pandas as pd
import pickle
import re
import datetime as dt
from dateutil.relativedelta import *
import time
import seaborn as sns

#SQL related - NEED TO DECIDE WHICH ONE I'LL BE USING AND DELETE THE REST
import sqlite3
import pandas.io.sql as pd_sql
# import psycopg2
# from sqlalchemy import create_engine

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 6000)

import warnings
warnings.filterwarnings('ignore')

In [3]:
def process_duplicates(new_ads_df):
    """Takes current dataframe (df) that needs to be cleaned up and a list of all ids that have at least one record where 
       listed_date is present. 
            This is a step-by-step guide of what this function does:
                1. Copies supplied dataframe
                2. Adds a column called "is_to_keep" which will be used to filter out unnecessary rows
                3. Iterates over the dataframe
                    i. Checks if a record had a price or status change, if so, it is kept (and we record the property id in a list for later)
                    ii. Checks if a record has complete duplicates in the dataset, if not, the record is kept
                    iii. Otherwise, moves on to check if:
                                    1. If it has a listed date column filled in
                                    2. If a record for the same property id has not been kept yet (not in kept_ids_list)
                                        If both are True, then the record will be kept
                    iv. Otherwise, moves on to check if:
                                    1. If the property id does not exist in the all_ids_with_listed_date list
                                    2. If a record for the same property id has not been kept yet (not in kept_ids_list)
                                        If both are True, then the record will be kept
        For all records that did not have a price or status change, the is_duplicate column is changed to False (since all duplicates will be dropped)
        Eventually columns no longer needed are dropped (listed_date_present, is_complete_duplicate and is_to_keep).                                
        Returns a new dataframe with only relevant columns (all complete duplicates removed)."""
    
    clean_df = df.copy()
    kept_ids_list = []
    clean_df['is_to_keep'] = False
    
    price_status_change_list = []
    
    for index, row in clean_df.iterrows():
        if row['is_price_change'] == True or row['is_status_change'] == True:
            clean_df.at[index, 'is_to_keep'] = True
            price_status_change_list.append(row['property_id'])
        elif row['is_complete_duplicate'] == False:
            clean_df.at[index, 'is_to_keep'] = True
            kept_ids_list.append(row['property_id'])
        elif row['listed_date_present'] == True and row['property_id'] not in kept_ids_list:
            clean_df.at[index, 'is_to_keep'] = True
            kept_ids_list.append(row['property_id'])
        elif row['property_id'] not in all_ids_with_listed_date and row['property_id'] not in kept_ids_list:
            clean_df.at[index, 'is_to_keep'] = True
            kept_ids_list.append(row['property_id'])
    
    # Reset the duplicate column to only say True if price or status change records remain in the dataset
    for index, row in clean_df.iterrows():
        if row['property_id'] not in price_status_change_list:
            clean_df.at[index, 'is_duplicate'] = False
            
    final_clean_df = clean_df[clean_df['is_to_keep'] == True]
    final_clean_df.drop(columns=['listed_date_present', 'is_complete_duplicate', 'is_to_keep'], inplace=True)    
    return final_clean_df


def string_to_date(value):
    """Whenever possible, converts text date value to date format (takes "Listen Since" field value as an argument)
    Returns the date value and a boolean True or False to identify whether it successfully converted the date or not."""
    try:
        listed_date = dt.datetime.strptime(value, '%B %d, %Y')
        listed_date = listed_date.date()
        return listed_date, True
    except:
        return np.NaN, False
    


def get_listed_date(value, scraped_date):
    """Converts a string in the 'Listed since' column into a listed_date value (in date format)"""
    today = dt.date.today()

    try:
        listed_date = dt.datetime.strptime(value, '%B %d, %Y').date()
    except:
        if scraped_date is np.NaN:
            listed_date = np.NaN    
        else:         
            if 'Today' in value:
                listed_date = scraped_date
            elif 'week' in value:
                weeks_listed = int(re.search('\d*', value).group())
                listed_date = scraped_date - relativedelta(weeks=weeks_listed)
            elif 'month' in value:
                months_listed = int(re.search('\d*', value).group())
                listed_date = scraped_date - relativedelta(months=months_listed)
            elif '6+' in value:
                # Not precise enough to tell, could be 7 months, could be 2 years
                listed_date = np.NaN
            else:
                listed_date = np.NaN
    return listed_date
    

def get_energy_label(value):
    """Takes 'Energy label' column and strips it from the words 'What does this mean?'
    Return just the label"""
    no_touch_list = ['Not required', 'Not available', np.NaN]
    if value not in no_touch_list:
        return value[0]
    else: 
        return np.NaN

def get_int(value):
    """Trims the price, area and other fields with numbers and converts them into int"""
    try:
        return re.sub('[€\sk.,m²m³v.o.n.permonthBeforeAfter]', '', value)
    except:
        return np.NaN
    

def get_rooms(value, room_type):
    """Retrieves the number of rooms specified by the type (room, bedroom, toilet, bathroom, etc.).
    If bedrooms are not specified and there is only 1 room - returns 0, if there is more than 1 room, but bedrooms not specified - returns NaN.
    Otherwise, returns the number of bedrooms"""
    try:
        value = value.lower()
    except:
        return np.NaN
    
    if room_type == 'room':
        try:
            return int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}'))
        except:
            return np.NaN
    
    if room_type == 'bedroom':
        try:
            if room_type not in value and int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}')) == 1:
                return 0
            elif room_type not in value:
                return np.NaN
            else:
                return int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}'))
        except:
            return np.NaN
    
    if room_type == 'toilet':
        try:
            if room_type not in value and int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}')) == 1:
                return 1
            elif room_type not in value:
                return np.NaN
            else:
                return int(re.search(f'\d*\s[a-z]*\s?{room_type}', value).group().strip(f' separate {room_type}'))
        except:
            return np.NaN
        
    if room_type == 'bathroom':
        try:
            if room_type not in value and int(re.search(f'\d*\s[a-z]*\s?toilet', value).group().strip(f' separate {room_type}')) == 1:
                return 1
            elif room_type not in value:
                return np.NaN
            else:
                return int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}'))
        except:
            return np.NaN
        
    
def get_bath_flag(value):
    """Takes Bathroom facilities column and create a Bath_Flag column if a bathtub / bath is available in the property.
    Returns True or False"""
    try:
        if 'bath' in value.lower():
            return True
        else:
            return False
    except:
        return np.NaN
    
    
def get_facilities(value, facility_type):
    """Take the Bathroom facilities column and facility type (toilet, shower, bath, jacuzzi, steam cabin, etc.)
    and returns the number of specified facilities"""
    try:
        value = value.lower()
        facility_type = facility_type.lower()
    except:
        return np.NaN
    
    try:
        return int(re.search(f'\d* {facility_type}', value).group().strip(f' {facility_type}'))
    except:
        if facility_type in value:
            return 1
        else:
            return 0

In [8]:
# First things first, connect to the postgresql database
conn = sqlite3.connect('./Database/ams_market_watch.db')  # You can create a new database by changing the name within the quotes
# cursor = conn.cursor()

# Now pull in existing records from the database table called funda_ads
current_data_df = pd.read_sql_query("""SELECT * FROM funda_ads""", con=conn)
original_ad_count = len(current_data_df)

# Also load the data from new_adverts.pkl file 
with open('./Cellar/new_adverts.pkl', 'rb') as new_ads_pkl:
    new_data = pickle.load(new_ads_pkl)

column_list = []

for ad in new_data:
    for feat_name in list(ad.keys()):
        if feat_name not in column_list:
            column_list.append(feat_name)  
    
# Step 3 - Add "Listed_date_present", "listed_date" and "is_duplicate" columns
column_list.append('listed_date')
column_list.append('listed_date_present')
column_list.append('is_duplicate')
column_list.append('is_complete_duplicate')
column_list.append('rank_same_record')


# Step 4 - Initiate the dataframe with the desired columns
new_data_df = pd.DataFrame(columns=column_list)


# Step 5 - Iterate over the ads_list and replace "Listed since" field with a date (using get_listed_date function), where possible 
    #i. #As part of this step, add another key-value pair to identify records with a successful conversion of "Listed since" to date
    #ii. Since we are iterating already, just add the "is_duplicate" column and set it to false
for ad in new_data:
    ad['listed_date'], ad['listed_date_present'] = string_to_date(ad['Listed since'])
    ad['is_duplicate'] = False
    ad['is_complete_duplicate'] = False
    ad['is_price_change'] = False
    ad['is_status_change'] = False
    
    
# Step 6 - iterate over the ads_list and append unique records to the empty dataframe (from step 2) 
    # i. Create a check_ads_list with only columns used to check for duplicates
    # ii. In the process check for duplicates and update "is_duplicate" value, if a record with the same:
        # property_id, title, asking price and status, has already been seen
check_duplicates = ['property_id', 'title']
check_complete_duplicate = ['property_id', 'title', 'Asking price', 'Status']
check_price_change = ['property_id', 'title', 'Status']
check_status_change = ['property_id', 'title', 'Asking price']
ads_seen_list = []

for ad in ads_list:
    # Keeps track of all the ads seen so far (dpl = duplicate)
    ad_check_dpl_value = [ad.get(key) for key in check_duplicates]
    ads_seen_list.append(ad_check_value)
    
    # Assign a rank_same_record value
    ad['rank_same_record'] = ads_seen_list.count(ad_check_value)
    
    # Create a list with columns cords for checking if a record is a complete duplicate (cdpl)
    ad_check_cdpl_value = [ad.get(key) for key in check_complete_duplicate]

    # Creates a smaller list of key values to check if a record already exists, but had a change in price or status (pc = price change, sc = status change)
    ad_check_pc_value = [ad.get(key) for key in check_price_change]
    ad_check_sc_value = [ad.get(key) for key in check_status_change]
    
    # Checks if the ad is in the ads_df already, this updates is_duplicate, is_price_change and is_status_change columns
    if ad_check_dpl_value in ads_df[check_duplicates].values.tolist():
        ad['is_duplicate'] = True
        # Now filter the dataframe to find all the records with the same id and title, and update their "is_duplicate" column value
        ads_df[(ads_df['property_id'] == ad_check_dpl_value[0]) & (ads_df['title'] == ad_check_dpl_value[1])]['is_duplicate'] = True
    if ad_check_cdpl_value in ads_df[check_complete_duplicate].values.tolist():
        ad['is_complete_duplicate'] = True
    if ad['is_complete_duplicate'] == False and ad_check_pc_value in ads_df[['property_id', 'title', 'Status']].values.tolist():
        ad['is_price_change'] = True
    if ad['is_complete_duplicate'] == False and ad_check_sc_value in ads_df[['property_id', 'title', 'Asking price']].values.tolist():
        ad['is_status_change'] = True    
        
    # Finally, append the advert into the dataframe
    new_data_df = new_data_df.append(ad, ignore_index=True)
    
    
# Convert columns to the right formats, clean up text values and make them numbers, etc.
new_data_df.drop(columns=['price'], inplace=True)
new_data_df['property_id'] = new_data_df['property_id'].apply(int)
new_data_df['listed_date'] = new_data_df.apply(lambda x: get_listed_date(x['Listed since'], x['scraped_date']), axis=1)
new_data_df['address'] = new_data_df['title']+', '+new_data_df['address']
new_data_df['Asking price'] = new_data_df['Asking price'].apply(get_int)
new_data_df['Asking price per m²'] = new_data_df['Asking price per m²'].apply(get_int)
new_data_df['VVE (Owners Association) contribution'] = new_data_df['VVE (Owners Association) contribution'].apply(get_int)
new_data_df['Year of construction'] = new_data_df['Year of construction'].apply(get_int)
new_data_df['Living area'] = new_data_df['Living area'].apply(get_int)
new_data_df['Exterior space attached to the building'] = new_data_df['Exterior space attached to the building'].apply(get_int)
new_data_df['Volume in cubic meters'] = new_data_df['Volume in cubic meters'].apply(get_int)
new_data_df['Rooms'] = new_data_df['Number of rooms'].apply(get_rooms, room_type='room')
new_data_df['Bedrooms'] = new_data_df['Number of rooms'].apply(get_rooms, room_type='bedroom')
new_data_df['Bathrooms'] = new_data_df['Number of bath rooms'].apply(get_rooms, room_type='bathroom')
new_data_df['Toilets'] = new_data_df['Number of bath rooms'].apply(get_rooms, room_type='toilet')
new_data_df['Has_Bathtub'] = new_data_df['Bathroom facilities'].apply(get_bath_flag)
new_data_df['Baths'] = new_data_df['Bathroom facilities'].apply(get_facilities, facility_type='bath')
new_data_df['Number of Toilets'] = new_data_df['Bathroom facilities'].apply(get_facilities, facility_type='toilet')
new_data_df['Showers'] = new_data_df['Bathroom facilities'].apply(get_facilities, facility_type='shower')
new_data_df['Energy label'] = new_data_df['Energy label'].apply(get_energy_label)
new_data_df['Provisional energy label'] = new_data_df['Provisional energy label'].apply(get_energy_label)


new_data_df.rename(columns={'Asking price': 'Asking price (€)', 'Asking price per m²': 'Asking price per m² (€)', 
                       'VVE (Owners Association) contribution': 'VVE contribution (monthly) (€)',
                      'Living area': 'Living area (m²)', 'Volume in cubic meters': 'Volume (m³)'}, inplace=True)

# Finally, update the funda_ads postgres table with the new records after removing unnecessary duplicates
final_new_df = process_duplicates(new_data_df)

updated_ad_count = pd.read_sql_query("""SELECT COUNT(*) FROM funda_ads""", con=conn)['COUNT(*)'][0]

print(f'Original table had {original_ad_count} ad records (non-unique, status and price changes are allowed). After the update\n
        the updated table has {updated_ad_count} advertisment records. In total {updated_ad_count - original_ad_count} records have been added.')

property_link  property_id  \
0    https://www.funda.nl/en/koop/amsterdam/apparte...     87214155   
1    https://www.funda.nl/en/koop/amsterdam/huis-41...     41738079   
2    https://www.funda.nl/en/koop/amsterdam/huis-41...     41642036   
3    https://www.funda.nl/en/koop/amsterdam/huis-41...     41866589   
4    https://www.funda.nl/en/koop/amsterdam/huis-41...     41865772   
5    https://www.funda.nl/en/koop/amsterdam/huis-41...     41854371   
6    https://www.funda.nl/en/koop/amsterdam/huis-41...     41851741   
7    https://www.funda.nl/en/koop/amsterdam/apparte...     41866585   
8    https://www.funda.nl/en/koop/amsterdam/apparte...     41866456   
9    https://www.funda.nl/en/koop/amsterdam/apparte...     87229843   
10   https://www.funda.nl/en/koop/amsterdam/apparte...     41865976   
11   https://www.funda.nl/en/koop/amsterdam/apparte...     41865424   
12   https://www.funda.nl/en/koop/amsterdam/apparte...     41853204   
13   https://www.funda.nl/en/koop/amsterdam/apparte...     41853276   
14   https://www.funda.nl/en/koop/amsterdam/apparte...     41853077   
15   https://www.funda.nl/en/koop/amsterdam/apparte...     41853883   
16   https://www.funda.nl/en/koop/amsterdam/apparte...     87217237   
17   https://www.funda.nl/en/koop/amsterdam/apparte...     41853438   
18   https://www.funda.nl/en/koop/amsterdam/huis-41...     41730485   
19   https://www.funda.nl/en/koop/amsterdam/apparte...     87037550   
20   https://www.funda.nl/en/koop/amsterdam/apparte...     87161621   
21   https://www.funda.nl/en/koop/amsterdam/apparte...     41851207   
22   https://www.funda.nl/en/koop/amsterdam/huis-41...     41851535   
23   https://www.funda.nl/en/koop/zwanenburg/appart...     41562106   
24   https://www.funda.nl/en/koop/amsterdam/apparte...     41852276   
25   https://www.funda.nl/en/koop/amsterdam/apparte...     41852868   
26   https://www.funda.nl/en/koop/amsterdam/apparte...     41851327   
27   https://www.funda.nl/en/koop/amsterdam/apparte...     87215922   
28   https://www.funda.nl/en/koop/amsterdam/apparte...     41851269   
29   https://www.funda.nl/en/koop/amsterdam/apparte...     41851013   
30   https://www.funda.nl/en/koop/amsterdam/apparte...     41851996   
31   https://www.funda.nl/en/koop/amsterdam/apparte...     41851471   
32   https://www.funda.nl/en/koop/amsterdam/apparte...     41851468   
33   https://www.funda.nl/en/koop/amsterdam/apparte...     87214699   
34   https://www.funda.nl/en/koop/amsterdam/apparte...     41850918   
35   https://www.funda.nl/en/koop/amsterdam/apparte...     41850863   
36   https://www.funda.nl/en/koop/amsterdam/apparte...     87214257   
37   https://www.funda.nl/en/koop/amsterdam/apparte...     87214201   
38   https://www.funda.nl/en/koop/amsterdam/apparte...     87213109   
39   https://www.funda.nl/en/koop/hoorn-nh/huis-870...     87092274   
40   https://www.funda.nl/en/koop/amsterdam/apparte...     87103127   
41   https://www.funda.nl/en/koop/amsterdam/apparte...     41857887   
42   https://www.funda.nl/en/koop/amsterdam/huis-41...     41854921   
43   https://www.funda.nl/en/koop/amsterdam/huis-41...     41853128   
44   https://www.funda.nl/en/koop/amsterdam/huis-41...     41853437   
45   https://www.funda.nl/en/koop/amsterdam/huis-41...     41851087   
46   https://www.funda.nl/en/koop/amsterdam/huis-87...     87215221   
47   https://www.funda.nl/en/koop/amsterdam/huis-41...     41859510   
48   https://www.funda.nl/en/koop/amsterdam/huis-41...     41858368   
49   https://www.funda.nl/en/koop/amsterdam/apparte...     41866570   
50   https://www.funda.nl/en/koop/amsterdam/apparte...     41865095   
51   https://www.funda.nl/en/koop/amsterdam/apparte...     41865726   
52   https://www.funda.nl/en/koop/amsterdam/apparte...     41865507   
53   https://www.funda.nl/en/koop/amsterdam/apparte...     41854918   
54   https://www.funda.nl/en/koop/amsterdam/apparte...     41854984   
55   https://www.funda.nl/en/koop/amsterdam/apparte...     418530